In [8]:
import pandas as pd
import numpy as np

In [145]:
df = pd.read_csv('testdata.csv')
data = np.array(df)
data

array([[156.81, 160.45, 156.36, 159.565, 'ok', 1637884800, 76959752],
       [160.24, 161.19, 158.7901, 159.37, 'ok', 1638144000, 88748217],
       [165.3, 165.52, 159.92, 159.985, 'ok', 1638230400, 174048056],
       [164.77, 170.3, 164.53, 167.48, 'ok', 1638316800, 152423003],
       [163.76, 164.2, 157.8, 158.735, 'ok', 1638403200, 136739174],
       [161.84, 164.96, 159.72, 164.02, 'ok', 1638489600, 118023116],
       [165.32, 167.8799, 164.28, 164.29, 'ok', 1638748800, 107496982],
       [171.18, 171.58, 168.34, 169.08, 'ok', 1638835200, 120405352],
       [175.08, 175.96, 170.7, 172.125, 'ok', 1638921600, 116998901],
       [174.56, 176.75, 173.92, 174.91, 'ok', 1639008000, 108923739],
       [179.45, 179.63, 174.69, 175.205, 'ok', 1639094400, 115402731],
       [175.74, 182.13, 175.53, 181.115, 'ok', 1639353600, 153237019],
       [174.33, 177.74, 172.21, 175.25, 'ok', 1639440000, 139380382],
       [179.3, 179.5, 172.3108, 175.11, 'ok', 1639526400, 131063257],
       [172.26, 1

# Relative Strength Index (RSI)

RSI is only calculated on the last 14 values

In [146]:
#calculating RSI
def RSI(data):
    gainavg = 0
    lossavg = 0
    for i in range(len(data)-14,len(data)):
        #difference of yesterdays and todays closing price
        #absolute value of losses (negative numbers when subtracting yesterdays closing from todays closing) is taken
        gain_loss = data[i][0]-data[i-1][0]
        if(gain_loss<=0):
            lossavg += gain_loss*-1
        else:
            gainavg += gain_loss
    gainavg/=14
    lossavg/=14
    #actual RSI
    return 100 - (100/(1+(gainavg/lossavg)))

In [147]:
RSI(data)

66.10887996430165

# Money Flow Index (MFI)

In [149]:
#calculating MFI
def MFI(data):
    count = 0
    pos_flow = 0
    neg_flow = 0
    for i in range(len(data)-14,len(data)):
        #money flow = (close+low+high)/3 * volume
        #determine if positive or negative money flow by whether the flow has gone up from the previous day or down
        current_flow = ((data[i][0]+data[i][1]+data[i][2])/3) * data[i][len(data[i])-1]
        prev_flow = ((data[i-1][0]+data[i-1][1]+data[i-1][2])/3) * data[i-1][len(data[i-1])-1]
        if(current_flow-prev_flow <= 0):
            neg_flow += current_flow
        else:
            pos_flow += current_flow
    #actual MFI
    return 100 - (100/(1+(pos_flow/neg_flow)))

In [150]:
MFI(data)

48.901415708322396

# Exponential Moving Average (EMA)

# Stochastic Oscillator

# Moving Average Convergence Divergence (MACD)